<a href="https://colab.research.google.com/github/hosseinmiripy/rezume/blob/main/%D9%BE%DB%8C%D8%AF%D8%A7%20%DA%A9%D8%B1%D8%AF%D9%86%20%D8%AA%D8%B1%D9%86%D8%AF%20%D9%87%D8%A7%DB%8C%20%D8%A7%DB%8C%D8%B1%D8%A7%D9%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
سیستم تحلیل بازار ایران - Market Analysis System for Iran
طراحی شده برای اجرای روزانه در Google Colab
"""
import os
os.environ['NUMPY_EXPERIMENTAL_ARRAY_FUNCTION'] = '0'
import sys
import warnings

warnings.filterwarnings('ignore')


# ===============================
# بخش 1: نصب ایمن کتابخانه‌ها
# ===============================

class SafePackageInstaller:
    """مدیریت نصب ایمن و مقاوم کتابخانه‌ها"""

    def __init__(self):
        self.installed_packages = []
        self.failed_packages = []

    def safe_install(self, package_name, pip_name=None, version=None):
        """نصب ایمن کتابخانه با مدیریت خطا"""
        if pip_name is None:
            pip_name = package_name

        try:
            # بررسی وجود کتابخانه
            __import__(package_name)
            print(f"✅ {package_name} در حال حاضر نصب است")
            self.installed_packages.append(package_name)
            return True
        except ImportError:
            try:
                # نصب کتابخانه
                install_cmd = f"{pip_name}"
                if version:
                    install_cmd += f"=={version}"

                print(f"📦 نصب {package_name}...")

                # اضافه کردن --no-cache-dir و --force-reinstall برای حل مشکل numpy
                result = os.system(f"pip install --quiet --no-cache-dir --force-reinstall {install_cmd}")

                if result != 0:
                    print(f"⚠️ تلاش مجدد برای نصب {package_name}...")
                    os.system(f"pip install --quiet --upgrade --no-deps {install_cmd}")

                # تست نصب
                __import__(package_name)
                print(f"✅ {package_name} با موفقیت نصب شد")
                self.installed_packages.append(package_name)
                return True

            except Exception as e:
                print(f"❌ خطا در نصب {package_name}: {str(e)}")
                self.failed_packages.append(package_name)
                return False

    def install_all_packages(self):
        """نصب تمام کتابخانه‌های مورد نیاز"""
        packages = [
            ('pandas', 'pandas'),
            ('numpy', 'numpy'),
            ('requests', 'requests'),
            ('bs4', 'beautifulsoup4'),  # اصلاح شده
            ('matplotlib', 'matplotlib'),
            ('seaborn', 'seaborn'),
            ('plotly', 'plotly'),
            ('sklearn', 'scikit-learn'),  # اصلاح شده
            ('textblob', 'textblob'),
            ('googletrans', 'googletrans==4.0.0rc1'),
            ('persiantools', 'persiantools'),
            ('khayyam', 'khayyam'),
            ('pytrends', 'pytrends'),
            ('telebot', 'pyTelegramBotAPI'),
            ('wordcloud', 'wordcloud')
            # hazm حذف شده تا مشکل numpy حل شود
        ]

        print("🚀 شروع نصب کتابخانه‌ها...")

        # ابتدا numpy را به‌روزرسانی کنیم
        print("🔧 به‌روزرسانی numpy...")
        os.system("pip install --quiet --upgrade numpy")

        for package_name, pip_name in packages:
            self.safe_install(package_name, pip_name)

        print(f"\n📊 خلاصه نصب:")
        print(f"✅ نصب شده: {len(self.installed_packages)}")
        print(f"❌ ناموفق: {len(self.failed_packages)}")

        if self.failed_packages:
            print(f"⚠️ کتابخانه‌های ناموفق: {', '.join(self.failed_packages)}")

# نصب کتابخانه‌ها
installer = SafePackageInstaller()
installer.install_all_packages()

# ===============================
# بخش 2: Import کتابخانه‌ها
# ===============================

# کتابخانه‌های پایه
import pandas as pd
import numpy as np
import json
import datetime
from datetime import datetime, timedelta
import time
import random
import re
from collections import defaultdict, Counter

# کتابخانه‌های تحلیل داده
try:
    from sklearn.cluster import KMeans
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.preprocessing import StandardScaler

    SKLEARN_AVAILABLE = True
except ImportError:
    SKLEARN_AVAILABLE = False
    print("⚠️ scikit-learn در دسترس نیست - تحلیل‌های پیشرفته غیرفعال شد")

# کتابخانه‌های ترسیم نمودار
# کتابخانه‌های ترسیم نمودار
try:
    import matplotlib.pyplot as plt
    import seaborn as sns

    plt.style.use('default')
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    print("⚠️ matplotlib در دسترس نیست - نمودارها غیرفعال شد")

# اصلاح import plotly
try:
    # تلاش برای import قدیمی
    import plotly.graph_objects as go

    # در صورت مشکل با plotly.express، از روش جایگزین استفاده کنیم
    try:
        import plotly.express as px
    except RuntimeError:
        print("⚠️ مشکل در plotly.express - استفاده از graph_objects")
        px = None

    PLOTLY_AVAILABLE = True
except ImportError:
    PLOTLY_AVAILABLE = False
    px = None
    go = None
    print("⚠️ plotly در دسترس نیست - نمودارهای تعاملی غیرفعال شد")
except RuntimeError as e:
    print(f"⚠️ مشکل plotly: {str(e)} - استفاده محدود")
    PLOTLY_AVAILABLE = False
    px = None
    go = None

# کتابخانه‌های دریافت داده
import requests

try:
    from bs4 import BeautifulSoup

    BS4_AVAILABLE = True
except ImportError:
    BS4_AVAILABLE = False
    print("⚠️ BeautifulSoup در دسترس نیست - کراولینگ وب غیرفعال شد")

try:
    from pytrends.request import TrendReq

    PYTRENDS_AVAILABLE = True
except ImportError:
    PYTRENDS_AVAILABLE = False
    print("⚠️ pytrends در دسترس نیست - Google Trends غیرفعال شد")

# کتابخانه‌های پردازش متن فارسی
try:
    from textblob import TextBlob
    from googletrans import Translator

    TEXT_PROCESSING_AVAILABLE = True
except ImportError:
    TEXT_PROCESSING_AVAILABLE = False
    print("⚠️ کتابخانه‌های پردازش متن در دسترس نیست")

try:
    import telebot

    TELEGRAM_AVAILABLE = True
except ImportError:
    TELEGRAM_AVAILABLE = False
    print("⚠️ telegram bot در دسترس نیست - ارسال گزارش غیرفعال شد")

# کتابخانه‌های تاریخ فارسی
try:
    import khayyam

    PERSIAN_DATE_AVAILABLE = True
except ImportError:
    PERSIAN_DATE_AVAILABLE = False
    print("⚠️ تاریخ فارسی در دسترس نیست - از تاریخ میلادی استفاده می‌شود")


class PersianTextProcessor:
    """پردازش‌کننده ساده متن فارسی (جایگزین hazm)"""

    @staticmethod
    def clean_persian_text(text):
        """پاک‌سازی ساده متن فارسی"""
        if not text:
            return text

        # حذف کاراکترهای اضافی
        text = re.sub(r'[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s\d\w]', '', text)

        # یکی کردن فاصله‌ها
        text = re.sub(r'\s+', ' ', text)

        return text.strip()

    @staticmethod
    def extract_keywords(text, max_keywords=10):
        """استخراج کلیدواژه‌های ساده"""
        if not text:
            return []

        # تقسیم به کلمات
        words = text.split()

        # فیلتر کلمات (حداقل 3 کاراکتر)
        words = [w for w in words if len(w) >= 3]

        # شمارش فراوانی
        word_count = Counter(words)

        # برگرداندن پرتکرارترین کلمات
        return [word for word, count in word_count.most_common(max_keywords)]
# ===============================
# بخش 3: تنظیمات اصلی
# ===============================

class Config:
    """تنظیمات سیستم"""

    # تنظیمات تلگرام (باید توسط کاربر تکمیل شود)
    TELEGRAM_BOT_TOKEN = "7962358346:AAHxMMHpgKfNZ8eUnvwKALDnct5eoXxqr6s"  # توکن ربات تلگرام
    TELEGRAM_CHAT_ID = "@andisharia_backup"  # شناسه چت

    # تنظیمات عمومی
    MAX_RETRIES = 3
    REQUEST_DELAY = 2  # تاخیر بین درخواست‌ها (ثانیه)

    # کلیدواژه‌های پایه برای تحلیل بازار ایران
    IRAN_KEYWORDS = [
        'موبایل', 'گوشی', 'لپ تاپ', 'کامپیوتر', 'تبلت',
        'لباس', 'کفش', 'کیف', 'ساعت', 'عطر',
        'کتاب', 'فیلم', 'بازی', 'موسیقی',
        'غذا', 'رستوران', 'آشپزی', 'نان',
        'خانه', 'اثاثیه', 'دکوراسیون', 'آشپزخانه',
        'ماشین', 'خودرو', 'موتور', 'دوچرخه',
        'سلامت', 'دارو', 'ورزش', 'تناسب اندام',
        'آرایش', 'زیبایی', 'مراقبت پوست',
        'سفر', 'گردشگری', 'هتل', 'بلیط',
        'تحصیل', 'کورس', 'زبان', 'مهارت'
    ]

    # دسته‌بندی‌های اصلی
    CATEGORIES = {
        'تکنولوژی': ['موبایل', 'گوشی', 'لپ تاپ', 'کامپیوتر', 'تبلت', 'هدفون'],
        'پوشاک': ['لباس', 'کفش', 'کیف', 'ساعت'],
        'زیبایی': ['آرایش', 'عطر', 'مراقبت پوست', 'زیبایی'],
        'خانه': ['اثاثیه', 'دکوراسیون', 'آشپزخانه', 'خانه'],
        'خودرو': ['ماشین', 'خودرو', 'موتور', 'دوچرخه'],
        'سلامت': ['سلامت', 'دارو', 'ورزش', 'تناسب اندام'],
        'سرگرمی': ['کتاب', 'فیلم', 'بازی', 'موسیقی'],
        'غذا': ['غذا', 'رستوران', 'آشپزی'],
        'سفر': ['سفر', 'گردشگری', 'هتل'],
        'آموزش': ['تحصیل', 'کورس', 'زبان', 'مهارت']
    }


# ===============================
# بخش 4: کلاس‌های اصلی سیستم
# ===============================

class PersianDateHelper:
    """کمک‌کننده تاریخ فارسی"""

    @staticmethod
    def get_persian_date():
        """دریافت تاریخ فارسی"""
        if PERSIAN_DATE_AVAILABLE:
            return khayyam.JalaliDatetime.now().strftime('%Y/%m/%d')
        else:
            return datetime.now().strftime('%Y/%m/%d')

    @staticmethod
    def get_persian_datetime():
        """دریافت تاریخ و زمان فارسی"""
        if PERSIAN_DATE_AVAILABLE:
            return khayyam.JalaliDatetime.now().strftime('%Y/%m/%d - %H:%M')
        else:
            return datetime.now().strftime('%Y/%m/%d - %H:%M')


class DataIngestion:
    """دریافت و جمع‌آوری داده"""

    def __init__(self):
        self.trends_data = {}
        self.products_data = []
        self.session = requests.Session()

        # تنظیمات Google Trends
        if PYTRENDS_AVAILABLE:
            try:
                self.pytrends = TrendReq(hl='fa-IR', tz=210)
            except:
                self.pytrends = None
                print("⚠️ خطا در اتصال به Google Trends")
        else:
            self.pytrends = None

    def get_google_trends(self, keywords, timeframe='today 7-d'):
        """دریافت داده‌های Google Trends"""
        if not self.pytrends or not keywords:
            return {}

        trends_data = {}

        try:
            # تقسیم کلیدواژه‌ها به گروه‌های 5 تایی (محدودیت Google)
            for i in range(0, len(keywords), 5):
                keyword_batch = keywords[i:i + 5]

                try:
                    self.pytrends.build_payload(
                        keyword_batch,
                        cat=0,
                        timeframe=timeframe,
                        geo='IR'  # ایران
                    )

                    # دریافت داده‌های ترند
                    interest_over_time = self.pytrends.interest_over_time()

                    if not interest_over_time.empty:
                        for keyword in keyword_batch:
                            if keyword in interest_over_time.columns:
                                trends_data[keyword] = {
                                    'current_score': interest_over_time[keyword].iloc[-1],
                                    'avg_score': interest_over_time[keyword].mean(),
                                    'trend_direction': self._calculate_trend_direction(interest_over_time[keyword]),
                                    'data': interest_over_time[keyword].tolist()
                                }

                    time.sleep(Config.REQUEST_DELAY)  # تاخیر برای جلوگیری از محدودیت

                except Exception as e:
                    print(f"⚠️ خطا در دریافت ترند برای {keyword_batch}: {str(e)}")
                    continue

        except Exception as e:
            print(f"❌ خطای کلی در Google Trends: {str(e)}")

        return trends_data

    def _calculate_trend_direction(self, data_series):
        """محاسبه جهت ترند (صعودی، نزولی، ثابت)"""
        if len(data_series) < 2:
            return 'ثابت'

        recent_avg = data_series.tail(3).mean()
        older_avg = data_series.head(3).mean()

        if recent_avg > older_avg * 1.2:
            return 'صعودی'
        elif recent_avg < older_avg * 0.8:
            return 'نزولی'
        else:
            return 'ثابت'

    def simulate_digikala_data(self):
        """شبیه‌سازی داده‌های دیجی‌کالا (تا زمان دسترسی به API واقعی)"""
        categories = list(Config.CATEGORIES.keys())
        products = []

        for category in categories:
            for i in range(5):  # 5 محصول برای هر دسته
                product = {
                    'name': f"محصول {i + 1} {category}",
                    'category': category,
                    'price': random.randint(100000, 5000000),
                    'rating': round(random.uniform(3.0, 5.0), 1),
                    'sales_count': random.randint(100, 10000),
                    'trend_score': random.randint(50, 100)
                }
                products.append(product)

        return products

    def collect_all_data(self):
        """جمع‌آوری تمام داده‌ها"""
        print("📊 شروع جمع‌آوری داده‌ها...")

        # دریافت داده‌های Google Trends
        print("🔍 دریافت Google Trends...")
        self.trends_data = self.get_google_trends(Config.IRAN_KEYWORDS)

        # دریافت داده‌های محصولات
        print("🛒 دریافت داده‌های محصولات...")
        self.products_data = self.simulate_digikala_data()

        print(f"✅ جمع‌آوری داده‌ها تکمیل شد:")
        print(f"   - {len(self.trends_data)} کلیدواژه ترند")
        print(f"   - {len(self.products_data)} محصول")

        return {
            'trends': self.trends_data,
            'products': self.products_data,
            'timestamp': datetime.now()
        }


class MarketAnalyzer:
    """تحلیل‌گر بازار"""

    def __init__(self, data):
        self.data = data
        self.analysis_results = {}

    def analyze_trends(self):
        """تحلیل ترندها"""
        trends = self.data.get('trends', {})

        # دسته‌بندی ترندها
        hot_trends = []  # ترندهای داغ
        rising_trends = []  # ترندهای صعودی
        falling_trends = []  # ترندهای نزولی

        for keyword, trend_data in trends.items():
            score = trend_data.get('current_score', 0)
            direction = trend_data.get('trend_direction', 'ثابت')

            if score > 80:
                hot_trends.append((keyword, score))

            if direction == 'صعودی':
                rising_trends.append((keyword, score))
            elif direction == 'نزولی':
                falling_trends.append((keyword, score))

        # مرتب‌سازی بر اساس امتیاز
        hot_trends = sorted(hot_trends, key=lambda x: x[1], reverse=True)[:10]
        rising_trends = sorted(rising_trends, key=lambda x: x[1], reverse=True)[:10]
        falling_trends = sorted(falling_trends, key=lambda x: x[1], reverse=True)[:10]

        return {
            'hot_trends': hot_trends,
            'rising_trends': rising_trends,
            'falling_trends': falling_trends,
            'total_keywords': len(trends)
        }

    def analyze_categories(self):
        """تحلیل دسته‌بندی‌ها"""
        trends = self.data.get('trends', {})
        products = self.data.get('products', [])

        category_analysis = {}

        for category, keywords in Config.CATEGORIES.items():
            # تحلیل ترند دسته‌بندی
            category_trends = []
            for keyword in keywords:
                if keyword in trends:
                    category_trends.append(trends[keyword]['current_score'])

            # تحلیل محصولات دسته‌بندی
            category_products = [p for p in products if p['category'] == category]

            avg_trend_score = np.mean(category_trends) if category_trends else 0
            avg_product_rating = np.mean([p['rating'] for p in category_products]) if category_products else 0
            total_sales = sum([p['sales_count'] for p in category_products])

            category_analysis[category] = {
                'trend_score': round(avg_trend_score, 2),
                'product_rating': round(avg_product_rating, 2),
                'total_sales': total_sales,
                'product_count': len(category_products),
                'status': self._get_category_status(avg_trend_score)
            }

        return category_analysis

    def _get_category_status(self, score):
        """تعیین وضعیت دسته‌بندی"""
        if score > 70:
            return 'عالی'
        elif score > 50:
            return 'خوب'
        elif score > 30:
            return 'متوسط'
        else:
            return 'ضعیف'

    def predict_opportunities(self):
        """پیش‌بینی فرصت‌های بازار"""
        trends_analysis = self.analyze_trends()
        category_analysis = self.analyze_categories()

        opportunities = []

        # فرصت‌های بر اساس ترندهای صعودی
        for keyword, score in trends_analysis['rising_trends'][:5]:
            opportunities.append({
                'type': 'ترند صعودی',
                'keyword': keyword,
                'score': score,
                'recommendation': f'سرمایه‌گذاری روی محصولات مرتبط با "{keyword}" توصیه می‌شود'
            })

        # فرصت‌های بر اساس دسته‌بندی‌های قوی
        strong_categories = [(cat, data) for cat, data in category_analysis.items()
                             if data['trend_score'] > 60 and data['status'] in ['عالی', 'خوب']]

        for category, data in strong_categories[:3]:
            opportunities.append({
                'type': 'دسته‌بندی قوی',
                'keyword': category,
                'score': data['trend_score'],
                'recommendation': f'توسعه محصولات در دسته "{category}" بازار مناسبی دارد'
            })

        return opportunities

    def run_full_analysis(self):
        """اجرای تحلیل کامل"""
        print("🧠 شروع تحلیل بازار...")

        self.analysis_results = {
            'trends_analysis': self.analyze_trends(),
            'category_analysis': self.analyze_categories(),
            'opportunities': self.predict_opportunities(),
            'analysis_timestamp': datetime.now()
        }

        print("✅ تحلیل بازار تکمیل شد")
        return self.analysis_results


class ReportGenerator:
    """تولیدکننده گزارش"""

    def __init__(self, analysis_results):
        self.analysis = analysis_results
        self.persian_date = PersianDateHelper.get_persian_datetime()

    def generate_daily_report(self):
        """تولید گزارش روزانه"""
        trends = self.analysis.get('trends_analysis', {})
        categories = self.analysis.get('category_analysis', {})
        opportunities = self.analysis.get('opportunities', [])

        report = f"""
🔥 گزارش روزانه بازار ایران
📅 تاریخ: {self.persian_date}

📈 ترندهای داغ امروز:
"""

        # ترندهای داغ
        hot_trends = trends.get('hot_trends', [])[:5]
        for i, (keyword, score) in enumerate(hot_trends, 1):
            report += f"{i}. {keyword} (امتیاز: {score})\n"

        report += f"\n🚀 ترندهای در حال رشد:\n"

        # ترندهای صعودی
        rising_trends = trends.get('rising_trends', [])[:5]
        for i, (keyword, score) in enumerate(rising_trends, 1):
            report += f"{i}. {keyword} (امتیاز: {score})\n"

        report += f"\n📊 وضعیت دسته‌بندی‌ها:\n"

        # بهترین دسته‌بندی‌ها
        sorted_categories = sorted(categories.items(),
                                   key=lambda x: x[1]['trend_score'], reverse=True)[:5]

        for category, data in sorted_categories:
            status_emoji = '🟢' if data['status'] == 'عالی' else '🟡' if data['status'] == 'خوب' else '🟠'
            report += f"{status_emoji} {category}: {data['trend_score']} ({data['status']})\n"

        report += f"\n💡 فرصت‌های امروز:\n"

        # فرصت‌ها
        for i, opp in enumerate(opportunities[:3], 1):
            report += f"{i}. {opp['recommendation']}\n"

        report += f"\n📋 خلاصه آمار:\n"
        report += f"• تعداد کلیدواژه‌های بررسی شده: {trends.get('total_keywords', 0)}\n"
        report += f"• تعداد دسته‌بندی‌های فعال: {len([c for c in categories.values() if c['trend_score'] > 30])}\n"
        report += f"• تعداد فرصت‌های شناسایی شده: {len(opportunities)}\n"

        return report

    def generate_weekly_report(self):
        """تولید گزارش هفتگی"""
        # برای سادگی، از گزارش روزانه استفاده می‌کنیم
        # در نسخه کامل، از داده‌های هفته قبل مقایسه خواهد شد
        daily_report = self.generate_daily_report()

        weekly_report = daily_report.replace('گزارش روزانه', 'گزارش هفتگی')
        weekly_report += f"\n\n📈 روند هفتگی:\n"
        weekly_report += "• این گزارش بر اساس تجمیع داده‌های هفته جاری تهیه شده است\n"
        weekly_report += "• برای مقایسه دقیق‌تر، سیستم به داده‌های بیشتری نیاز دارد\n"

        return weekly_report

    def generate_monthly_report(self):
        """تولید گزارش ماهانه"""
        daily_report = self.generate_daily_report()

        monthly_report = daily_report.replace('گزارش روزانه', 'گزارش ماهانه')
        monthly_report += f"\n\n📊 تحلیل ماهانه:\n"
        monthly_report += "• این گزارش بر اساس روندهای ماه جاری تهیه شده است\n"
        monthly_report += "• برای تحلیل دقیق‌تر ماهانه، سیستم به 30 روز داده نیاز دارد\n"

        return monthly_report

    def create_chart(self):
        """ایجاد نمودار (در صورت امکان)"""
        if not MATPLOTLIB_AVAILABLE:
            print("⚠️ matplotlib در دسترس نیست - نمودار ایجاد نمی‌شود")
            return None

        try:
            categories = self.analysis.get('category_analysis', {})

            if not categories:
                print("⚠️ داده‌ای برای نمودار موجود نیست")
                return None

            # داده‌های نمودار
            cat_names = list(categories.keys())
            scores = [categories[cat]['trend_score'] for cat in cat_names]

            # ایجاد نمودار با تنظیمات بهتر
            plt.figure(figsize=(12, 6))

            # استفاده از font که در colab موجود است
            plt.rcParams.update({'font.size': 10})

            bars = plt.bar(cat_names, scores, color='skyblue', alpha=0.7)

            # تنظیمات نمودار
            plt.title('Market Categories Analysis - Iran', fontsize=14, pad=20)
            plt.ylabel('Trend Score')
            plt.xticks(rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.3)

            # اضافه کردن مقادیر روی میله‌ها
            for bar, score in zip(bars, scores):
                plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1,
                         f'{score:.1f}', ha='center', va='bottom')

            plt.tight_layout()

            # ذخیره نمودار
            chart_path = '/content/market_chart.png'
            plt.savefig(chart_path, dpi=300, bbox_inches='tight')
            plt.close()  # بستن figure برای آزادسازی حافظه

            print("✅ نمودار ایجاد شد")
            return chart_path

        except Exception as e:
            print(f"⚠️ خطا در ایجاد نمودار: {str(e)}")
            return None

# !/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
سیستم تحلیل بازار ایران - بخش دوم
ادامه کلاس TelegramSender و سیستم اصلی
"""


class TelegramSender:
    """ارسال‌کننده تلگرام"""

    def __init__(self, token, chat_id):
        self.token = token
        self.chat_id = chat_id
        self.bot = None

        if TELEGRAM_AVAILABLE and token != "YOUR_BOT_TOKEN_HERE":
            try:
                self.bot = telebot.TeleBot(token)
                print("✅ اتصال به تلگرام برقرار شد")
            except Exception as e:
                print(f"❌ خطا در اتصال به تلگرام: {str(e)}")

    def send_report(self, report_text, chart_path=None):
        """ارسال گزارش به تلگرام"""
        if not self.bot:
            print("⚠️ ربات تلگرام تنظیم نشده است")
            print("📝 گزارش:")
            print(report_text)
            return False

        try:
            # تقسیم متن به بخش‌های کوچکتر (محدودیت تلگرام: 4096 کاراکتر)
            max_length = 4000
            text_parts = []

            if len(report_text) > max_length:
                # تقسیم متن
                lines = report_text.split('\n')
                current_part = ""

                for line in lines:
                    if len(current_part + line + '\n') > max_length:
                        if current_part:
                            text_parts.append(current_part.strip())
                        current_part = line + '\n'
                    else:
                        current_part += line + '\n'

                if current_part:
                    text_parts.append(current_part.strip())
            else:
                text_parts = [report_text]

            # ارسال بخش‌های متن
            for i, part in enumerate(text_parts):
                if i == 0:
                    sent_message = self.bot.send_message(
                        chat_id=self.chat_id,
                        text=part
                    )
                else:
                    sent_message = self.bot.send_message(
                        chat_id=self.chat_id,
                        text=f"📄 ادامه گزارش ({i + 1}):\n\n{part}"
                    )

                time.sleep(1)  # تاخیر کوتاه بین پیام‌ها

            # ارسال نمودار (در صورت وجود)
            if chart_path and os.path.exists(chart_path):
                try:
                    with open(chart_path, 'rb') as photo:
                        self.bot.send_photo(
                            chat_id=self.chat_id,
                            photo=photo,
                            caption="📊 نمودار تحلیل بازار ایران"
                        )
                    print("✅ نمودار با موفقیت ارسال شد")
                except Exception as e:
                    print(f"⚠️ خطا در ارسال نمودار: {str(e)}")

            print("✅ گزارش با موفقیت ارسال شد")
            return True

        except Exception as e:
            print(f"❌ خطا در ارسال گزارش: {str(e)}")
            return False

    def test_connection(self):
        """تست اتصال به تلگرام"""
        if not self.bot:
            return False

        try:
            me = self.bot.get_me()
            print(f"✅ اتصال موفق - نام ربات: {me.first_name}")
            return True
        except Exception as e:
            print(f"❌ خطا در تست اتصال: {str(e)}")
            return False


class MarketAnalysisSystem:
    """سیستم اصلی تحلیل بازار ایران"""

    def __init__(self, telegram_token=None, telegram_chat_id=None):
        """راه‌اندازی سیستم"""
        print("🚀 راه‌اندازی سیستم تحلیل بازار ایران...")

        # تنظیمات تلگرام
        self.telegram_token = telegram_token or Config.TELEGRAM_BOT_TOKEN
        self.telegram_chat_id = telegram_chat_id or Config.TELEGRAM_CHAT_ID

        # اجزای سیستم
        self.data_ingester = DataIngestion()
        self.telegram_sender = TelegramSender(self.telegram_token, self.telegram_chat_id)

        # متغیرهای داخلی
        self.raw_data = None
        self.analysis_results = None
        self.reports = {}

        print("✅ سیستم آماده است")

    def check_system_status(self):
        """بررسی وضعیت سیستم"""
        print("\n🔍 بررسی وضعیت سیستم:")

        status = {
            'google_trends': PYTRENDS_AVAILABLE,
            'text_processing': TEXT_PROCESSING_AVAILABLE,
            'charts': MATPLOTLIB_AVAILABLE,
            'machine_learning': SKLEARN_AVAILABLE,
            'telegram': TELEGRAM_AVAILABLE and self.telegram_token != "YOUR_BOT_TOKEN_HERE",
            'web_scraping': BS4_AVAILABLE
        }

        for component, available in status.items():
            status_emoji = "✅" if available else "❌"
            print(f"   {status_emoji} {component}")

        # هشدارهای مهم
        if not status['telegram']:
            print("\n⚠️ هشدار: تلگرام تنظیم نشده - گزارش‌ها در کنسول نمایش داده می‌شود")

        if not status['google_trends']:
            print("\n⚠️ هشدار: Google Trends در دسترس نیست - از داده‌های شبیه‌سازی شده استفاده می‌شود")

        return status

    def collect_data(self):
        """جمع‌آوری داده‌ها"""
        print("\n📊 مرحله 1: جمع‌آوری داده‌ها")
        self.raw_data = self.data_ingester.collect_all_data()
        return self.raw_data

    def analyze_market(self):
        """تحلیل بازار"""
        if not self.raw_data:
            print("❌ ابتدا داده‌ها را جمع‌آوری کنید")
            return None

        print("\n🧠 مرحله 2: تحلیل بازار")
        analyzer = MarketAnalyzer(self.raw_data)
        self.analysis_results = analyzer.run_full_analysis()
        return self.analysis_results

    def generate_reports(self):
        """تولید گزارش‌ها"""
        if not self.analysis_results:
            print("❌ ابتدا تحلیل بازار را انجام دهید")
            return None

        print("\n📝 مرحله 3: تولید گزارش‌ها")

        report_generator = ReportGenerator(self.analysis_results)

        # تولید گزارش‌های مختلف
        self.reports = {
            'daily': report_generator.generate_daily_report(),
            'weekly': report_generator.generate_weekly_report(),
            'monthly': report_generator.generate_monthly_report()
        }

        # ایجاد نمودار
        chart_path = report_generator.create_chart()
        self.reports['chart_path'] = chart_path

        print("✅ گزارش‌ها تولید شدند")
        return self.reports

    def send_reports(self, report_type='daily'):
        """ارسال گزارش‌ها"""
        if not self.reports:
            print("❌ ابتدا گزارش‌ها را تولید کنید")
            return False

        print(f"\n📤 مرحله 4: ارسال گزارش {report_type}")

        if report_type not in self.reports:
            print(f"❌ نوع گزارش '{report_type}' موجود نیست")
            return False

        success = self.telegram_sender.send_report(
            self.reports[report_type],
            self.reports.get('chart_path')
        )

        return success

    def run_daily_analysis(self):
        """اجرای تحلیل روزانه کامل"""
        print("=" * 50)
        print("🇮🇷 سیستم تحلیل بازار ایران - اجرای روزانه")
        print("=" * 50)

        # بررسی وضعیت سیستم
        system_status = self.check_system_status()

        try:
            # مراحل تحلیل
            self.collect_data()
            self.analyze_market()
            self.generate_reports()

            # ارسال گزارش روزانه
            success = self.send_reports('daily')

            if success:
                print("\n🎉 تحلیل روزانه با موفقیت تکمیل شد!")
            else:
                print("\n⚠️ تحلیل انجام شد اما ارسال گزارش با مشکل مواجه شد")

            return True

        except Exception as e:
            print(f"\n❌ خطا در اجرای تحلیل: {str(e)}")
            return False

    def run_weekly_analysis(self):
        """اجرای تحلیل هفتگی"""
        print("=" * 50)
        print("🇮🇷 سیستم تحلیل بازار ایران - اجرای هفتگی")
        print("=" * 50)

        try:
            self.collect_data()
            self.analyze_market()
            self.generate_reports()

            success = self.send_reports('weekly')

            if success:
                print("\n🎉 تحلیل هفتگی با موفقیت تکمیل شد!")

            return success

        except Exception as e:
            print(f"\n❌ خطا در اجرای تحلیل هفتگی: {str(e)}")
            return False

    def run_monthly_analysis(self):
        """اجرای تحلیل ماهانه"""
        print("=" * 50)
        print("🇮🇷 سیستم تحلیل بازار ایران - اجرای ماهانه")
        print("=" * 50)

        try:
            self.collect_data()
            self.analyze_market()
            self.generate_reports()

            success = self.send_reports('monthly')

            if success:
                print("\n🎉 تحلیل ماهانه با موفقیت تکمیل شد!")

            return success

        except Exception as e:
            print(f"\n❌ خطا در اجرای تحلیل ماهانه: {str(e)}")
            return False

    def interactive_menu(self):
        """منوی تعاملی برای کاربر"""
        while True:
            print("\n" + "=" * 40)
            print("🇮🇷 سیستم تحلیل بازار ایران")
            print("=" * 40)
            print("1️⃣ تحلیل روزانه")
            print("2️⃣ تحلیل هفتگی")
            print("3️⃣ تحلیل ماهانه")
            print("4️⃣ بررسی وضعیت سیستم")
            print("5️⃣ تست اتصال تلگرام")
            print("6️⃣ نمایش گزارش در کنسول")
            print("0️⃣ خروج")
            print("-" * 40)

            try:
                choice = input("انتخاب شما: ").strip()

                if choice == '1':
                    self.run_daily_analysis()
                elif choice == '2':
                    self.run_weekly_analysis()
                elif choice == '3':
                    self.run_monthly_analysis()
                elif choice == '4':
                    self.check_system_status()
                elif choice == '5':
                    self.telegram_sender.test_connection()
                elif choice == '6':
                    self.show_console_report()
                elif choice == '0':
                    print("👋 خداحافظ!")
                    break
                else:
                    print("❌ انتخاب نامعتبر")

            except KeyboardInterrupt:
                print("\n👋 خداحافظ!")
                break
            except Exception as e:
                print(f"❌ خطا: {str(e)}")

    def show_console_report(self):
        """نمایش گزارش در کنسول"""
        if not self.reports:
            print("⚠️ ابتدا یکی از تحلیل‌ها را اجرا کنید")
            return

        print("\n📊 کدام گزارش نمایش داده شود؟")
        print("1. روزانه")
        print("2. هفتگی")
        print("3. ماهانه")

        choice = input("انتخاب: ").strip()

        if choice == '1' and 'daily' in self.reports:
            print("\n" + "=" * 50)
            print(self.reports['daily'])
            print("=" * 50)
        elif choice == '2' and 'weekly' in self.reports:
            print("\n" + "=" * 50)
            print(self.reports['weekly'])
            print("=" * 50)
        elif choice == '3' and 'monthly' in self.reports:
            print("\n" + "=" * 50)
            print(self.reports['monthly'])
            print("=" * 50)
        else:
            print("❌ گزارش موجود نیست")


# ===============================
# بخش 5: تنظیمات کاربر و اجرا
# ===============================

def setup_telegram_credentials():
    """تنظیم اطلاعات تلگرام"""
    print("\n🔧 تنظیم اطلاعات تلگرام:")
    print("برای دریافت گزارش‌ها در تلگرام، لطفاً اطلاعات زیر را وارد کنید:")
    print("(در صورت عدم تمایل، Enter بزنید)")

    token = input("Bot Token: ").strip()
    chat_id = input("Chat ID: ").strip()

    if token and chat_id:
        return token, chat_id
    else:
        print("⚠️ اطلاعات تلگرام وارد نشد - گزارش‌ها در کنسول نمایش داده می‌شود")
        return None, None


def quick_demo():
    """نمایش سریع سیستم"""
    print("\n🎬 اجرای نمایشی سیستم...")

    # ایجاد سیستم با تنظیمات پیش‌فرض
    system = MarketAnalysisSystem()

    # اجرای تحلیل روزانه
    success = system.run_daily_analysis()

    if success:
        print("\n✅ نمایش تکمیل شد!")

        # نمایش خلاصه گزارش در کنسول
        if system.reports and 'daily' in system.reports:
            print("\n📊 خلاصه گزارش روزانه:")
            print("-" * 30)

            # نمایش 10 خط اول گزارش
            report_lines = system.reports['daily'].split('\n')[:10]
            for line in report_lines:
                print(line)

            print("...")
            print(f"(نمایش {len(report_lines)} خط از {len(system.reports['daily'].split(chr(10)))} خط کل)")


def main():
    """تابع اصلی"""
    print("🇮🇷 خوش آمدید به سیستم تحلیل بازار ایران!")
    print("\nآیا می‌خواهید:")
    print("1️⃣ تنظیمات کامل و اجرای تعاملی")
    print("2️⃣ اجرای سریع نمایشی")
    print("3️⃣ فقط تحلیل روزانه")

    choice = input("\nانتخاب شما: ").strip()

    if choice == '1':
        # تنظیمات کامل
        token, chat_id = setup_telegram_credentials()
        system = MarketAnalysisSystem(token, chat_id)
        system.interactive_menu()

    elif choice == '2':
        # نمایش سریع
        quick_demo()

    elif choice == '3':
        # فقط تحلیل روزانه
        token, chat_id = setup_telegram_credentials()
        system = MarketAnalysisSystem(token, chat_id)
        system.run_daily_analysis()

    else:
        print("❌ انتخاب نامعتبر")


# ===============================
# اجرای سیستم
# ===============================

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n👋 خداحافظ!")
    except Exception as e:
        print(f"\n❌ خطای کلی: {str(e)}")
        print("🔄 لطفاً دوباره تلاش کنید")

# ===============================
# نمونه استفاده سریع
# ===============================

# برای اجرای سریع، خطوط زیر را غیرفعال کنید:
"""
# ایجاد سیستم
system = MarketAnalysisSystem(
    telegram_token="YOUR_BOT_TOKEN",  # توکن ربات خود را اینجا قرار دهید
    telegram_chat_id="YOUR_CHAT_ID"  # شناسه چت خود را اینجا قرار دهید
)

# اجرای تحلیل روزانه
system.run_daily_analysis()
"""

print("✅ سیستم تحلیل بازار ایران آماده است!")
print("📖 برای شروع، تابع main() را اجرا کنید")

🚀 شروع نصب کتابخانه‌ها...
🔧 به‌روزرسانی numpy...
✅ pandas در حال حاضر نصب است
✅ numpy در حال حاضر نصب است
✅ requests در حال حاضر نصب است
✅ bs4 در حال حاضر نصب است
✅ matplotlib در حال حاضر نصب است
✅ seaborn در حال حاضر نصب است
✅ plotly در حال حاضر نصب است
✅ sklearn در حال حاضر نصب است
✅ textblob در حال حاضر نصب است
✅ googletrans در حال حاضر نصب است
✅ persiantools در حال حاضر نصب است
✅ khayyam در حال حاضر نصب است
✅ pytrends در حال حاضر نصب است
✅ telebot در حال حاضر نصب است
✅ wordcloud در حال حاضر نصب است

📊 خلاصه نصب:
✅ نصب شده: 15
❌ ناموفق: 0
🇮🇷 خوش آمدید به سیستم تحلیل بازار ایران!

آیا می‌خواهید:
1️⃣ تنظیمات کامل و اجرای تعاملی
2️⃣ اجرای سریع نمایشی
3️⃣ فقط تحلیل روزانه

انتخاب شما: 1

🔧 تنظیم اطلاعات تلگرام:
برای دریافت گزارش‌ها در تلگرام، لطفاً اطلاعات زیر را وارد کنید:
(در صورت عدم تمایل، Enter بزنید)
Bot Token: 7962358346:AAHxMMHpgKfNZ8eUnvwKALDnct5eoXxqr6s
Chat ID: @andisharia_backup
🚀 راه‌اندازی سیستم تحلیل بازار ایران...
✅ اتصال به تلگرام برقرار شد
✅ سیستم آماده است

🇮🇷 سیستم ت